## Pages List (Buy)

In [1]:
#import requests
#import regex
#from bs4 import BeautifulSoup

#pages_list = []
#for i in range(1, 838):
    #page = requests.get('https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|'+ str(i) +'||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||')
    #pages_list.append(page)

## Properties Pages List (Buy)

In [ ]:
import requests
import regex
from bs4 import BeautifulSoup

pages_list = []
for i in range(0, 21):
    pages_list.append(requests.get('https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|'+ str(i) +'||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||'))
print()
soup = BeautifulSoup(page.content, 'html.parser')
division = soup.find(id='divAdverts')
properties_pages_list = []
for i in range(0, 34):
    properties = division.find(id='rowIndex_' + str(i))
    property = ''
    if (i == 0):
        property = properties.find('li', class_='media')
        property = property.find('span').attrs.get('onclick')
    else:
        property = properties.find('li', class_='media').attrs.get('onclick')
    url_property = str(property).replace('javascript:window.location=', '')
    url_property = 'https://www.fincaraiz.com.co' + url_property.replace("'", "")
    print('Property #' + str(i) + ": " + str(url_property))
    properties_pages_list.append(url_property)